In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4273708771977138024
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14343274496
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13145887071927280808
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
!pip install torch
!pip install torchvision
!pip install scikit-learn
!pip install scipy
!pip install ekphrasis
!pip install openai-clip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-clip: filename=open

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import clip
import torch.optim as optim
import time
from sklearn import metrics
import argparse
import numpy as np
from torch import nn
from torch.nn import functional as F
import torch
import sys
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Insert the directory
sys.path.insert(0,'/content/drive/MyDrive/MamiGit/mami')
import random
from helper_functions import *
from text_normalizer import *

In [5]:
import os
os.getcwd()

'/content'

In [6]:
## Dataset Loading
tr_df = pd.read_csv('/content/drive/MyDrive/MamiGit/mami/data/train.tsv', sep='\t') # use '/t' for main training data
vl_df = pd.read_csv('/content/drive/MyDrive/MamiGit/mami/data/validation.tsv', sep='\t') # use '/t' for main training data
ts_df = pd.read_csv('/content/drive/MyDrive/MamiGit/mami/data/test.tsv', sep='\t') # use '/t' for main training data
#tr_df = tr_df.head(300)
#vl_df = vl_df.head(300)
#ts_df = ts_df.head(50)

In [7]:
print(tr_df.head())

   file_name  label  shaming  stereotype  objectification  violence  \
0   8716.jpg      1        0           1                1         0   
1   3066.jpg      1        1           1                0         0   
2   6038.jpg      0        0           0                0         0   
3  10861.jpg      1        0           0                1         0   
4  11198.jpg      0        0           0                0         0   

                                                text  
0  GETS MARRIED TO GIRL OF HIS DREAMS SHE DOESN'T...  
1  When your mama don't change yo diaper for 19 y...  
2  Some people want a big house, a fast car, and ...  
3     FAP FAP FAP FAP FAP FAP memecenter Meme Center  
4              I RAISE. A I CALL. I FOLD. I'M ALL IN  


In [8]:
tr_df.rename(columns = {'Text Transcription':'text'}, inplace = True)
tr_df.rename(columns = {'misogynous':'label'}, inplace = True)

my_list = tr_df.columns.values.tolist()
print (my_list)

['file_name', 'label', 'shaming', 'stereotype', 'objectification', 'violence', 'text']


In [9]:
print(vl_df.head())

  file_name  label  shaming  stereotype  objectification  violence  \
0  1377.jpg      1        0           1                0         0   
1  5498.jpg      1        0           1                0         0   
2  5697.jpg      1        0           1                0         0   
3  5182.jpg      1        0           0                0         0   
4  6162.jpg      1        0           1                0         0   

                                                text  
0  IF WOMEN WANT EQUAL PAY THEY SHOULD DO EQUAL W...  
1                  IF A WOMAN WERE TO DRIVE YOUR CAR  
2                           When a woman is a driver  
3  SEXY BITCHES? Ba GIVE THEM BACON TO MAKE THEM ...  
4  When you reach peak feminism FEMINIST Just Bec...  


In [10]:
print(ts_df.head)

<bound method NDFrame.head of      file_name  label  shaming  stereotype  objectification  violence  \
0    15236.jpg      0        0           0                0         0   
1    15805.jpg      1        0           1                1         0   
2    16254.jpg      0        0           0                0         0   
3    16191.jpg      1        0           1                1         0   
4    15952.jpg      0        0           0                0         0   
..         ...    ...      ...         ...              ...       ...   
995  15591.jpg      1        0           1                1         0   
996  15049.jpg      0        0           0                0         0   
997  15363.jpg      1        0           1                1         0   
998  15199.jpg      0        0           0                0         0   
999  15853.jpg      0        0           0                0         0   

                                                  text  
0    FACEBOOK SINGLES GROUPS BELIKE 

In [11]:
#seed = 42
#random.seed(seed)
#np.random.seed(seed)
#torch.manual_seed(seed)
#torch.cuda.manual_seed_all(seed)

In [12]:
# Arguments
batch_size=64
init_lr=1e-4
epochs=20
max_length=77
lr=1e-4


In [13]:
import clip
import torch
from torch import nn

# Pre-trained Stream Models
clip_nms = {'vit32': 'ViT-B/32', 'vit16': 'ViT-B/16', 'rn50': 'RN50', 'rn504': 'RN50x4', 'rn101': 'RN101', 'rn5016': 'RN50x16', 'rn5064': 'RN50x64', 'vit14': 'ViT-L/14'}
clip_dim = {'vit32': 512, 'vit16': 512, 'vit14': 768, 'rn50': 1024, 'rn504': 640, 'rn101': 512, 'rn5016': 768, 'rn5064': 1024}

vmodel='vit14' # Choose the desired model

# Load the pre-trained CLIP model
clip_model, _ = clip.load(clip_nms[vmodel], jit=False)
input_resolution = clip_model.visual.input_resolution

# Set the model to float and evaluation mode
clip_model = clip_model.float().eval()

# Wrap the model with DataParallel for multi-GPU training if available
clip_model = nn.DataParallel(clip_model)

# Get the dimension for the chosen model
dim = clip_dim[vmodel]

100%|████████████████████████████████████████| 890M/890M [00:08<00:00, 104MiB/s]


In [14]:
from torchvision import transforms

input_resolution = 224  # Specify the desired input resolution

transform_config = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_resolution, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.RandomHorizontalFlip(),
        transforms.RandomGrayscale(p=0.1),
        transforms.RandomRotation(degrees=10),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.48145466, 0.4578275, 0.40821073],
            std=[0.26862954, 0.26130258, 0.27577711]
        )
    ]),
    'test': transforms.Compose([
        transforms.Resize((input_resolution, input_resolution), interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.48145466, 0.4578275, 0.40821073],
            std=[0.26862954, 0.26130258, 0.27577711]
        )
    ])
}

In [15]:
# !pip install simpletokenizer
from clip.simple_tokenizer import SimpleTokenizer
import torch

_tokenizer = SimpleTokenizer()

def tokenize(text, context_length: int = 77):
    sot_token = _tokenizer.encoder["<|startoftext|>"]
    eot_token = _tokenizer.encoder["<|endoftext|>"]

    tokens = [sot_token] + _tokenizer.encode(text)[:context_length-2] + [eot_token]
    result = torch.zeros(context_length, dtype=torch.long)
    mask = torch.zeros(context_length, dtype=torch.long)
    result[:len(tokens)] = torch.tensor(tokens)
    mask[:len(tokens)] = 1

    return result, mask

In [16]:
train_data = CustomDatasetFixed(tr_df, 'training', transform_config['test'], preprocess, tokenize, max_length)
valid_data = CustomDatasetFixed(vl_df, 'training', transform_config['test'], preprocess, tokenize, max_length)
test_data = CustomDatasetFixed(ts_df, 'test', transform_config['test'], preprocess, tokenize, max_length)

In [17]:
import torch.nn as nn

class MMNetwork(nn.Module):
    def __init__(self, vdim, tdim, n_cls):
        super(MMNetwork, self).__init__()

        self.vfc = nn.Linear(vdim, 512)
        self.tfc = nn.Linear(vdim, 512)

        self.pre_output_layer1 = nn.Linear(512, 1024)
        self.pre_output_layer2 = nn.Linear(1024, 64)

        self.cf1 = nn.Linear(64, 1)
        self.cf2 = nn.Linear(64, 1)
        self.cf3 = nn.Linear(64, 1)
        self.cf4 = nn.Linear(64, 1)
        self.cf5 = nn.Linear(64, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, vx, tx, masks=None):
        fi = vx
        ft = tx
        
        pi = self.dropout(self.relu(self.vfc(fi)))
        pt = self.dropout(self.relu(self.tfc(ft)))
        fim = pi * pt

        output = self.dropout(self.relu(self.pre_output_layer1(fim)))
        output = self.dropout(self.relu(self.pre_output_layer2(output)))

        output1 = torch.sigmoid(self.dropout(self.relu(self.cf1(output))))
        output2 = torch.sigmoid(self.dropout(self.relu(self.cf2(output))))
        output3 = torch.sigmoid(self.dropout(self.relu(self.cf3(output))))
        output4 = torch.sigmoid(self.dropout(self.relu(self.cf4(output))))
        output5 = torch.sigmoid(self.dropout(self.relu(self.cf5(output))))

        return output1, output2, output3, output4, output5

In [26]:
def performErrorHandling(all_preds1, all_labels1):

  with open('/content/drive/MyDrive/non_matching_indexes_20_1e4.txt', 'w') as f:
            # Loop through the indexes of the labels in the first list
            for i in range(len(all_labels1)): 
              # Check if the label in the first list does not match the label in the second list
              if all_labels1[i] != all_preds1[i]:
                # Write the index to the file
                f.write(str(i) + '\n')

In [19]:
def evaluate(model, loader, isTest):
    model.eval()
    test_loss = 0
    all_preds1 = []
    all_labels1 = []
    all_preds2 = []
    all_labels2 = []
    all_preds3 = []
    all_labels3 = []
    all_preds4 = []
    all_labels4 = []
    all_preds5 = []
    all_labels5 = []

    with torch.no_grad():
        for img_inps, txt_tokens, masks, labels1, labels2, labels3, labels4, labels5 in loader:

            img_inps, txt_tokens, masks, labels1, labels2, labels3, labels4, labels5 = img_inps.to(device), \
                txt_tokens.to(device), masks.to(device), labels1.to(device), labels2.to(device), labels3.to(device), \
                labels4.to(device), labels5.to(device)

            img_feats = clip_model.module.encode_image(img_inps)
            txt_feats = clip_model.module.encode_text(txt_tokens)

            outputs1, outputs2, outputs3, outputs4, outputs5 = model(img_feats, txt_feats, masks)

            preds1 = (outputs1>0.5).int()
            preds2 = (outputs2>0.5).int()
            preds3 = (outputs3>0.5).int()
            preds4 = (outputs4>0.5).int()
            preds5 = (outputs5>0.5).int()
            
            test_loss += (criterion(outputs1, labels1.unsqueeze(1).float()).item() + criterion(outputs2, labels2.unsqueeze(1).float()).item() + \
                            criterion(outputs3, labels3.unsqueeze(1).float()).item() + criterion(outputs4, labels4.unsqueeze(1).float()).item() + \
                            criterion(outputs5, labels5.unsqueeze(1).float()).item())

            all_preds1.extend(preds1.cpu().numpy().flatten())
            all_labels1.extend(labels1.cpu().numpy().flatten())
            all_preds2.extend(preds2.cpu().numpy().flatten())
            all_labels2.extend(labels2.cpu().numpy().flatten())
            all_preds3.extend(preds3.cpu().numpy().flatten())
            all_labels3.extend(labels3.cpu().numpy().flatten())
            all_preds4.extend(preds4.cpu().numpy().flatten())
            all_labels4.extend(labels4.cpu().numpy().flatten())
            all_preds5.extend(preds5.cpu().numpy().flatten())
            all_labels5.extend(labels5.cpu().numpy().flatten())

        acc = metrics.accuracy_score(all_labels1, all_preds1)
        f1_1 = metrics.f1_score(all_labels1, all_preds1, average='macro')
        f1_2 = metrics.f1_score(all_labels2, all_preds2, average='micro')
        f1_3 = metrics.f1_score(all_labels3, all_preds3, average='micro')
        f1_4 = metrics.f1_score(all_labels4, all_preds4, average='micro')
        f1_5 = metrics.f1_score(all_labels5, all_preds5, average='micro')

        if(isTest):
          print("Performing Error Handling Next - ")
          performErrorHandling(all_preds1, all_labels1)

    return test_loss/len(loader), acc, f1_1, f1_2, f1_3, f1_4, f1_5

In [20]:
from torch.utils.data import DataLoader, Dataset, sampler
train_loader = DataLoader(train_data, shuffle=True, num_workers=2, batch_size=batch_size)
valid_loader = DataLoader(valid_data, num_workers=2, batch_size=batch_size)
test_loader = DataLoader(test_data, num_workers=2, batch_size=batch_size)

In [21]:
def train(model, optimizer, lr_scheduler, num_epochs):
    since = time.time()

    best_model = model
    best_acc = 0.0
    best_val_loss = 100
    best_epoch = 0
    best_f1 = 0

    for epoch in range(1, num_epochs + 1):
        print(f"Epoch {epoch}/{num_epochs}")
        print("-" * 10)

        since2 = time.time()

        model.train()

        running_loss = 0.0
        running_corrects = 0
        total_samples = 0.0
        cnt = 0

        for img_inps, txt_tokens, masks, labels1, labels2, labels3, labels4, labels5 in tr_loader:
            img_inps, txt_tokens, masks, labels1, labels2, labels3, labels4, labels5 = (
                img_inps.to(device),
                txt_tokens.to(device),
                masks.to(device),
                labels1.to(device),
                labels2.to(device),
                labels3.to(device),
                labels4.to(device),
                labels5.to(device),
            )

            optimizer.zero_grad()

            with torch.no_grad():
                img_feats = clip_model.module.encode_image(img_inps)
                txt_feats = clip_model.module.encode_text(txt_tokens)

            outputs1, outputs2, outputs3, outputs4, outputs5 = model(
                img_feats, txt_feats, masks
            )
            preds1 = (outputs1 > 0.5).int()

            loss = criterion(outputs1, labels1.unsqueeze(1).float()) + criterion(
                outputs2, labels2.unsqueeze(1).float()
            ) + criterion(outputs3, labels3.unsqueeze(1).float()) + criterion(
                outputs4, labels4.unsqueeze(1).float()
            ) + criterion(outputs5, labels5.unsqueeze(1).float())

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            running_loss += loss.item()
            running_corrects += torch.sum(preds1 == labels1.data.view_as(preds1)).item()
            total_samples += len(labels1)

            if cnt % 40 == 0:
                print(
                    f"[{epoch}, {cnt+1}] loss: {loss.item():.4f}, Acc: {(100.0 * running_corrects) / total_samples:.2f}"
                )

            cnt += 1

        if scheduler:
            lr_scheduler.step()

        train_loss = running_loss / len(tr_loader)
        train_acc = running_corrects / len(tr_loader.dataset)

        print(
            f"Training Loss: {train_loss:.6f} Acc: {100.0 * train_acc:.2f}"
        )

        test_loss, test_acc, test_f1_1, test_f1_2, test_f1_3, test_f1_4, test_f1_5 = evaluate(
            model, vl_loader, False
        )

        print(
            f"Epoch: {epoch}, Val Loss: {test_loss:.4f}, Acc: {test_acc*100:.2f}, F1_1: {test_f1_1*100:.2f}, F1_2: {test_f1_2*100:.2f}, F1_3: {test_f1_3*100:.2f}, F1_4: {test_f1_4*100:.2f}, F1_5: {test_f1_5*100:.2f}"
        )

        # deep copy the model
        if test_f1_1 >= best_f1:
            best_acc = test_acc
            best_val_loss = test_loss
            best_model = copy.deepcopy(model)
            best_epoch = epoch
            best_f1 = test_f1_1

    time_elapsed2 = time.time() - since2
    print('Epoch complete in {:.0f}m {:.0f}s'.format(
        time_elapsed2 // 60, time_elapsed2 % 60))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return best_model, best_epoch


In [22]:
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
dim = clip_dim[vmodel]
# Data Loaders
tr_loader = DataLoader(train_data, shuffle=True, num_workers=4, batch_size=batch_size)
vl_loader = DataLoader(valid_data, num_workers=2, batch_size=batch_size)
ts_loader = DataLoader(test_data, num_workers=2, batch_size=batch_size)

# Model
model = MMNetwork(dim, dim, 1).to(device)
print(model)

optimizer = optim.Adam(model.parameters(), init_lr, betas=(0.99, 0.98), weight_decay=1e-4)
criterion = nn.BCELoss()
num_train_steps = len(train_data) // batch_size * epochs
num_warmup_steps = int(0.1 * num_train_steps)
print(num_train_steps, num_warmup_steps)  # Print Number of total and warmup steps

scheduler = MultiStepLR(optimizer, milestones=[5, 10, 15], gamma=0.5)
import time
start_time = time.time()
model_ft, best_epoch = train(model, optimizer, scheduler, num_epochs=epochs)
print("--- %s seconds ---" % (time.time() - start_time))

# torch.save(model_ft.state_dict(), 'saved_models/trained_model_%s.pt'%(args.net, vmodel))

vl_loss, vl_acc, vl_f1, _, _, _, _ = evaluate(model_ft, vl_loader, False)
print('Validation best epoch: %d, Val Loss: %.4f, ACC: %.2f, F1: %.2f' % (best_epoch, np.round(vl_loss, 4), vl_acc*100, vl_f1*100))

MMNetwork(
  (vfc): Linear(in_features=768, out_features=512, bias=True)
  (tfc): Linear(in_features=768, out_features=512, bias=True)
  (pre_output_layer1): Linear(in_features=512, out_features=1024, bias=True)
  (pre_output_layer2): Linear(in_features=1024, out_features=64, bias=True)
  (cf1): Linear(in_features=64, out_features=1, bias=True)
  (cf2): Linear(in_features=64, out_features=1, bias=True)
  (cf3): Linear(in_features=64, out_features=1, bias=True)
  (cf4): Linear(in_features=64, out_features=1, bias=True)
  (cf5): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (softmax): Softmax(dim=1)
)
2800 280
Epoch 1/20
----------
[1, 1] loss: 3.4827, Acc: 54.69
[1, 41] loss: 3.4650, Acc: 53.47
[1, 81] loss: 3.4387, Acc: 55.77
[1, 121] loss: 3.3400, Acc: 58.52
Training Loss: 3.434830 Acc: 60.18
Epoch: 1, Val Loss: 3.3473, Acc: 75.30, F1_1: 74.34, F1_2: 87.70, F1_3: 72.40, F1_4: 77.20, F1_5: 90.00
Epoch 2/20
----------
[2,

In [27]:
ts_loss, ts_acc, ts_f1, ts_f2, ts_f3, ts_f4, ts_f5 = evaluate(model_ft, test_loader, True)
taskB_F1 = (ts_f2 + ts_f3 + ts_f4 + ts_f5)/4
print('Test results:, Test Loss: %.4f, ACC: %.2f, F1- Task A: %.2f, F1 - Task B: %.2f'%(np.round(ts_loss,4), ts_acc*100, ts_f1*100, taskB_F1*100))

Performing Error Handling Next - 
Test results:, Test Loss: 3.5288, ACC: 75.20, F1- Task A: 74.23, F1 - Task B: 76.38
